In [16]:
import http.client
import json
import pandas as pd



class Anymarket:
    def __init__(
        self, credentials_path: str, base_url: str = "/v2/products"
    ):
        self.credentials_path = credentials_path

    def connect(self, base_url) -> json:
        try:
            with open(self.credentials_path, 'r') as file:
                credentials = json.load(file)
                conn = http.client.HTTPConnection("sandbox-api.anymarket.com.br")

                headers = {
                    'Content-Type': "application/json",
                    'gumgaToken': credentials['anymarket_token'],
                    }

                conn.request("GET", base_url, headers=headers)

                return conn.getresponse()
        
        except Exception as e:
            raise Exception(f'Failed to connect: {e}')

    def get_products_quantity(self) -> int:
        try:
            conector = self.connect('/v2/products')
            data = conector.read()
            json_data = json.loads(data.decode("utf-8"))

            # Acesse o valor de 'totalElements' e guarde em uma variável
            total_elements = json_data['page']['totalElements']

            return total_elements
        except Exception as e:
            raise Exception(f'Failed to connect: {e}')

    def get_all_products(self, quantity:int) -> pd.DataFrame:
        try:
            conector = self.connect(f'/v2/products?limit={quantity}')
            data = conector.read()
            json_data = json.loads(data.decode("utf-8"))

            products = json_data.get('content', [])
            partner_ids = [sku.get('partnerId', '') for product in products for sku in product.get('skus', [])]

            product_ids = [product['id'] for product in json_data.get('content', [])]
            print(json_data)
            return partner_ids, product_ids
        except Exception as e:
            raise Exception(f'Failed to connect: {e}')
        
    def change_data_patch(self, list: list):
        try:
            for id in list:


                conn = http.client.HTTPConnection("sandbox-api.anymarket.com.br")

                payload = "{\n  \"calculatedPrice\": false,\n  \"definitionPriceScope\": \"SKU_MARKETPLACE\"\n}"

                headers = {
                    'Content-Type': "application/merge-patch+json",
                    'gumgaToken': "32293584L39959642E1789758617689C169644661768900O891.I"
                }

                conn.request("PATCH", f"/v2/products/{id}", payload, headers)

                res = conn.getresponse()
                data = res.read()

                print(data.decode("utf-8"))

        except Exception as e:
            print(e)

    def query_all_products(self, id_list: list):
        try:
            result_list = []

            for product_id in id_list:
                conn = http.client.HTTPConnection("sandbox-api.anymarket.com.br")

                headers = {
                    'Content-Type': "application/json",
                    'gumgaToken': "32293584L39959642E1789758617689C169644661768900O891.I"
                }

                conn.request("GET", f"/v2/skus/marketplaces?partnerID={product_id}", headers=headers)

                res = conn.getresponse()
                data = res.read()
                json_data = json.loads(data.decode("utf-8"))
                result_list.extend(json_data)

            # Criar um DataFrame a partir de uma lista de dicionários
            df = pd.json_normalize(result_list)
            
            print(df.columns)

            # Renomear a coluna 'skuInMarketplace' para 'sku (*)'
            df.rename(columns={'skuInMarketplace': 'sku (*)'}, inplace=True)

            # Selecionar as colunas desejadas
            
            # Remover coluna vazia
            df.dropna(subset=['fields.title'], inplace=True)

            return df

        except Exception as e:
            print(e)

    def change_price(self, marketplace: str, df: pd.DataFrame):
        try:
            
            df_beleza_na_web = df.loc[df['marketPlace'] == marketplace]
            print(df_beleza_na_web)
            for index, row in df_beleza_na_web.iterrows():
                conn = http.client.HTTPConnection("sandbox-api.anymarket.com.br")

                payload = "[\n  {\n    \"id\": 3349911,\n    \"price\": 50,\n    \"discountPrice\": 50\n  }\n]"


                headers = {
                    'Content-Type': "application/json",
                    'gumgaToken': "32293584L39959642E1789758617689C169644661768900O891.I"
                }

                conn.request("PUT", "/v2/skus/marketplaces", payload, headers)

                res = conn.getresponse()
                data = res.read()

                print(data.decode("utf-8"))

        except Exception as e:
            print(e)
    

In [34]:
am = Anymarket('../credentials/k_service_account_credentials.json')

In [35]:
quantity = am.get_products_quantity()

In [36]:
all_products = am.get_all_products(quantity)

{'links': [{'rel': 'next', 'href': 'http://api.anymarket.com.br/v2/products?limit=13&offset=13'}], 'content': [{'id': 3349911, 'title': 'Moroccanoil Tratamento Óleo Original 25ml', 'category': {'id': 893146, 'name': 'Tratamento de Cabelo', 'path': 'Beleza e Cuidado com Cabelo/Tratamento de Cabelo'}, 'origin': {'id': 0, 'description': 'NATIONAL'}, 'model': 'Original', 'gender': 'UNISSEX', 'warrantyTime': 0, 'warrantyText': '0', 'priceFactor': 5, 'calculatedPrice': False, 'definitionPriceScope': 'SKU_MARKETPLACE', 'hasVariations': False, 'isProductActive': True, 'skus': [{'id': 2223132, 'title': 'Moroccanoil Tratamento Óleo Original 25ml', 'partnerId': 'MO0001', 'ean': '7290011521127', 'price': 49.9, 'amount': 163, 'additionalTime': 0, 'stockLocalId': 10244}], 'allowAutomaticSkuMarketplaceCreation': True}, {'id': 3349993, 'title': 'Amend Leave In Millenar Óleos Egípcios Balm Selante 180g', 'description': '<p>teste</p>', 'category': {'id': 893146, 'name': 'Tratamento de Cabelo', 'path': '

In [39]:
change = am.change_data_patch(all_products[1])

{"id":3349911,"title":"Moroccanoil Tratamento Óleo Original 25ml","category":{"id":893146,"name":"Tratamento de Cabelo","path":"Beleza e Cuidado com Cabelo/Tratamento de Cabelo"},"origin":{"id":0,"description":"NATIONAL"},"model":"Original","gender":"UNISSEX","warrantyTime":0,"warrantyText":"0","priceFactor":5.0,"calculatedPrice":false,"definitionPriceScope":"SKU_MARKETPLACE","hasVariations":false,"isProductActive":true,"skus":[{"id":2223132,"title":"Moroccanoil Tratamento Óleo Original 25ml","partnerId":"MO0001","ean":"7290011521127","price":49.9,"sellPrice":49.9,"amount":163,"additionalTime":0,"stockLocalId":10244,"active":true}],"allowAutomaticSkuMarketplaceCreation":true,"type":"SIMPLE"}
{"id":3349993,"title":"Amend Leave In Millenar Óleos Egípcios Balm Selante 180g","description":"<p>teste</p>","category":{"id":893146,"name":"Tratamento de Cabelo","path":"Beleza e Cuidado com Cabelo/Tratamento de Cabelo"},"origin":{"id":0,"description":"NATIONAL"},"warrantyTime":0,"warrantyText":"

In [40]:
query = am.query_all_products(all_products[0])


Index(['id', 'accountName', 'idAccount', 'marketPlace', 'index',
       'publicationStatus', 'transmissionStatus', 'marketplaceStatus', 'price',
       'priceFactor', 'discountPrice', 'skuInMarketplace', 'isCatalog',
       'fields.is_main_sku', 'fields.DISCOUNT_VALUE', 'fields.title',
       'fields.DISCOUNT_TYPE', 'fields.HAS_DISCOUNT', 'fields.priceFactor'],
      dtype='object')


In [41]:
query

,sku (*),id,marketPlace,publicationStatus,marketplaceStatus,price,fields.title
0,AM0011,81315,ECOMMERCE,ACTIVE,Ativo Dessincronizado,50,Amend Leave In Millenar Óleos Egípcios Balm Se...
1,AM0049,81316,ECOMMERCE,ACTIVE,Ativo Dessincronizado,50,Amend Leave In Millenar Óleos Indianos Balm Se...


  sku (*)     id marketPlace publicationStatus      marketplaceStatus  price  \
0  AM0011  81315   ECOMMERCE            ACTIVE  Ativo Dessincronizado     50   
1  AM0049  81316   ECOMMERCE            ACTIVE  Ativo Dessincronizado     50   

                                        fields.title  
0  Amend Leave In Millenar Óleos Egípcios Balm Se...  
1  Amend Leave In Millenar Óleos Indianos Balm Se...  


In [43]:
change_price = am.change_price('ECOMMERCE', query)

  sku (*)     id marketPlace publicationStatus      marketplaceStatus  price  \
0  AM0011  81315   ECOMMERCE            ACTIVE  Ativo Dessincronizado     50   
1  AM0049  81316   ECOMMERCE            ACTIVE  Ativo Dessincronizado     50   

                                        fields.title  
0  Amend Leave In Millenar Óleos Egípcios Balm Se...  
1  Amend Leave In Millenar Óleos Indianos Balm Se...  
{"code":"422","message":"Método PUT não suportado para path solicitado. Verifique a documentação para maiores detalhes."}


[Errno 104] Connection reset by peer


In [56]:
conn = http.client.HTTPConnection("sandbox-api.anymarket.com.br")

payload = "[\n  {\n    \"id\": 81315,\n    \"price\": 110,\n    \"discountPrice\": 101\n  }\n]"

headers = {
    'Content-Type': "application/json",
    'gumgaToken': "32293584L39959642E1789758617689C169644661768900O891.I"
}

conn.request("PUT", "/v2/skus/marketplaces", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"code":"422","message":"Método PUT não suportado para path solicitado. Verifique a documentação para maiores detalhes."}


In [55]:
import http.client

conn = http.client.HTTPConnection("sandbox-api.anymarket.com.br")

payload = "[\n  {\n    \"id\": 81315,\n    \"price\": 120,\n    \"discountPrice\": 110\n  }\n]"

headers = {
    'Content-Type': "application/json",
    'gumgaToken': "32293584L39959642E1789758617689C169644661768900O891.I"
    }

conn.request("PUT", "/v2/skus/marketplaces/prices", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

[]


In [45]:
conn = http.client.HTTPConnection("sandbox-api.anymarket.com.br")

headers = {
    'Content-Type': "application/json",
    'gumgaToken': "32293584L39959642E1789758617689C169644661768900O891.I"
}

conn.request("GET", f"/v2/skus/marketplaces?partnerID=AM0011", headers=headers)

res = conn.getresponse()
data = res.read()
json_data = json.loads(data.decode("utf-8"))


In [46]:
json_data

[{'id': 81315,
  'accountName': 'ECOMMERCE',
  'idAccount': 7800,
  'marketPlace': 'ECOMMERCE',
  'index': 1,
  'publicationStatus': 'ACTIVE',
  'transmissionStatus': 'OK',
  'marketplaceStatus': 'Ativo Dessincronizado',
  'price': 50,
  'priceFactor': 2.431907,
  'discountPrice': 40,
  'skuInMarketplace': 'AM0011',
  'fields': {'is_main_sku': 'true',
   'DISCOUNT_VALUE': '20',
   'title': 'Amend Leave In Millenar Óleos Egípcios Balm Selante 180g',
   'DISCOUNT_TYPE': 'PERCENT',
   'HAS_DISCOUNT': 'true',
   'priceFactor': '0'},
  'isCatalog': False}]